In [ ]:
%pip install mojito2 python-dotenv pandas pymysql mysql-connector-python sqlalchemy

In [5]:
import os
from dotenv import load_dotenv
import datetime

load_dotenv()

KIS_APP_KEY = os.getenv("KIS_APP_KEY")
KIS_APP_SECRET = os.getenv("KIS_APP_SECRET")
KIS_ACC_NO = os.getenv("KIS_ACC_NO")
DB_HOST = os.getenv("DB_HOST")
DB_USERNAME = os.getenv("DB_USERNAME")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEME = os.getenv("DB_SCHEME")

In [6]:
import pandas as pd
import requests as rq
from io import StringIO

def get_krx_otp(otp_params, headers):
  krx_gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
  krx_otp = rq.post(krx_gen_otp_url, otp_params, headers = headers).text
  return krx_otp

def download_krx_sector(at_date, market = None):
  if not market:
    kospi_df = download_krx_sector(at_date, 'KOSPI')
    kosdaq_df = download_krx_sector(at_date, 'KOSDAQ')
    return pd.concat([kospi_df, kosdaq_df])

  headers = {             
    "User-Agent": "Mozilla/5.0",
    "Referer": "http://data.krx.co.kr/"
  }
  otp_params = {
      'locale': 'ko_KR',
      'trdDd': at_date,
      'money': '1',
      'csvxls_isNo': 'false',
      'name': 'fileDown',
      'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
  }
  if market == 'KOSPI':
    otp_params |= { 'mktId': 'STK' }
  elif market == 'KOSDAQ':
    otp_params |= { 'mktId': 'KSQ', 'segTpCd': 'ALL' }

  otp = get_krx_otp(otp_params, headers)

  download_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
  download_params = {
      'code': otp
  }
  res = rq.post(download_url, download_params, headers = headers)
  res.encoding = 'euc-kr'
  return pd.read_csv(StringIO(res.text))

In [7]:
# 현재 날짜를 YYYYMMDD 형식으로 변환
now = datetime.datetime.now()
current_date_str = now.strftime('%Y%m%d')

df = download_krx_sector(current_date_str)

In [8]:
df

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률


In [9]:
df_column_mapping = {
    '단축코드': 'SHORT_CODE',
    # '서브카테고리' : 'SUB_CATEGORY_ID',
    '한글 종목약명': 'STOCK_NAME',
    '표준코드': 'STANDARD_CODE',
    '시장구분': 'STOCK_EXCHANGE_MARKET',
    '상장주식수': 'MARKET_CAPITALIZATION',
    '시가': 'OPEN_PRICE',
    '종가': 'CLOSED_PRICE',
    '고가': 'HIGH_PRICE',
    '저가': 'LOW_PRICE',
    '거래정지': 'STOCK_TRADE_STATUS',
}

# 컬럼명 매핑 적용
df.rename(columns=df_column_mapping, inplace=True)

In [10]:
df

,종목코드,종목명,CLOSED_PRICE,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률


In [11]:
for column in [
 'SHORT_CODE',
 'SUB_CATEGORY_ID',
 'STOCK_NAME',
 'STANDARD_CODE',
 'STOCK_EXCHANGE_MARKET',
 'MARKET_CAPITALIZATION',
 'OPEN_PRICE',
 'CLOSED_PRICE',
 'HIGH_PRICE',
 'LOW_PRICE',
 'STOCK_TRADE_STATUS']:
    if column not in df.columns:
        df[column] = 1


In [12]:

columns_to_keep = [
 'SHORT_CODE',
 'SUB_CATEGORY_ID',
 'STOCK_NAME',
 'STANDARD_CODE',
 'STOCK_EXCHANGE_MARKET',
 'MARKET_CAPITALIZATION',
 'OPEN_PRICE',
 'CLOSED_PRICE',
 'HIGH_PRICE',
 'LOW_PRICE',
 'STOCK_TRADE_STATUS'
]

df = df[columns_to_keep]

In [13]:
df['STOCK_EXCHANGE_MARKET'] = df['STOCK_EXCHANGE_MARKET'].map({'KOSPI': "0", 'KOSDAQ': "1"})

In [14]:
df.to_csv("stock_info.csv", index=True, encoding="utf-8-sig")


In [16]:
from sqlalchemy import create_engine
# MySQL 데이터베이스 연결 설정
db_username = DB_USERNAME
db_password = DB_PASSWORD
db_host = DB_HOST
db_name = DB_SCHEME

# SQLAlchemy 엔진 생성
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}')

df.to_sql(name='STOCK', con=engine, if_exists='append', index=False)


0